# U.S. Geological Survey Class GW3099
Advanced Modeling of Groundwater Flow (GW3099)\
Boise, Idaho\
September 16 - 20, 2024

![title](../../images/ClassLocation.jpg)

## NetCDF input
  - Running `MODFLOW 6` in validate mode offers a convenient way to export existing gridded package input arrays to `NetCDF`
  - These exports can then be used as simulation inputs

In [ ]:
import pathlib as pl

import flopy
import matplotlib.pyplot as plt
import xarray as xa
import xugrid as xu

### Load the base watershed model
  - Use the base watershed model to demonstrate `MODFLOW 6 NetCDF` export and input functionality

In [ ]:
name = "watershed"
sim = flopy.mf6.MFSimulation.load(
    sim_name=name, sim_ws=pl.Path("../../data/watershed/")
)

### Modify the model to export ascii input arrays to NetCDF output
  - Define `export_netcdf` in the groundwater flow model `name_file`, specifying `structured` as the output type
  - Activate `DIS` package array export with the `EXPORT_ARRAY_NETCDF` keyword in the package `OPTION` block

In [ ]:
gwf = sim.get_model()

In [ ]:
gwf.export_netcdf = "structured"
gwf.dis.export_array_netcdf = True

### Run the model in validate mode
  - Change simulation workspace to structured subdirectory
  - Write model files
  - Run the model with `--mode` option set to `validate`

In [ ]:
ws = pl.Path("temp/step1/structured")
sim.set_sim_path(ws)

In [ ]:
sim.write_simulation()

In [ ]:
cargs = ["--mode", "validate"]
sim.run_simulation(cargs=cargs)

### Visualize export data
  - Load the structured NetCDF file with `xarray`
  - Visualize `botm` layers 1 and 5

In [ ]:
nc_ds = xa.open_dataset(ws / "model.nc")

In [ ]:
nc_ds

In [ ]:
nc_ds["dis_botm"].isel(z=0).plot()
plt.gca().set_aspect(1.0)
plt.show(block=False)

In [ ]:
nc_ds["dis_botm"].isel(z=4).plot()
plt.gca().set_aspect(1.0)
plt.show(block=False)

### Update model to run with NetCDF array inputs
  - Set `start_date_time` in the TDIS file
  - Deactivate `NetCDF` exports for model
  - Activate 'NetCDF` import for model

In [ ]:
# set simulation start_date_time
sim.tdis.start_date_time = "1980-01-01"
gwf.export_netcdf = None
gwf.nc_filerecord = "model.nc"

### Write the simulation
  - Overwrite `DIS` package to specify gridded arrays should be read from `NetCDF`

In [ ]:
sim.write_simulation()

In [ ]:
with open(ws / "model.dis", "w") as f:
    f.write("BEGIN options\n")
    f.write("  XORIGIN       0.00000000\n")
    f.write("  YORIGIN       0.00000000\n")
    f.write("END options\n\n")
    f.write("BEGIN dimensions\n")
    f.write("  NLAY  5\n")
    f.write("  NROW  41\n")
    f.write("  NCOL  73\n")
    f.write("END dimensions\n\n")
    f.write("BEGIN griddata\n")
    f.write("  delr NETCDF\n")
    f.write("  delc NETCDF\n")
    f.write("  top NETCDF\n")
    f.write("  botm NETCDF\n")
    f.write("  idomain NETCDF\n")
    f.write("END griddata\n\n")

with open(ws / "model.dis", "r") as f:
    print(f.read())

### Run the simulation

In [ ]:
sim.run_simulation()

### Update model to export input arrays to `ugrid` `NetCDF`
  - Set the `export_netcdf` variable in the model name file to `ugrid` to export input arrays as mesh NetCDF output
  - Deactive NetCDF as input
  - Turn off DIS array export and turn on IC array export

In [ ]:
gwf.name_file.export_netcdf = "ugrid"
gwf.nc_filerecord = None
gwf.dis.export_array_netcdf = False
gwf.ic.export_array_netcdf = True

### Run the model in validate mode
  - Change simulation workspace to structured subdirectory
  - Write model files
  - Run the model with `--mode` option set to `validate`

In [ ]:
ws = pl.Path("temp/step1/mesh")
sim.set_sim_path(ws)

In [ ]:
sim.write_simulation()

In [ ]:
cargs = ["--mode", "validate"]
sim.run_simulation(cargs=cargs)

### Visualize export data
  - Load the structured NetCDF file with `xugrid`
  - Visualize `strt` layer 1

In [ ]:
nc_ds = xu.open_dataset(ws / "model.nc")

In [ ]:
nc_ds

In [ ]:
nc_ds["ic_strt_l1"].isel().ugrid.plot()
plt.gca().set_aspect(1.0)
plt.show(block=False)

### Update model to run with NetCDF array inputs
  - Deactivate `NetCDF` exports for model
  - Activate 'NetCDF` import for model

In [ ]:
# update model for netcdf input
gwf.export_netcdf = None
gwf.nc_filerecord = "model.nc"

### Write the simulation
  - Overwrite `IC` package to specify gridded arrays should be read from `NetCDF`

In [ ]:
sim.write_simulation()

In [ ]:
with open(ws / "model.ic", "w") as f:
    f.write("BEGIN options\n")
    f.write("END options\n\n")
    f.write("BEGIN griddata\n")
    f.write("  strt NETCDF\n")
    f.write("END griddata\n\n")

with open(ws / "model.ic", "r") as f:
    print(f.read())

### Run the simulation

In [ ]:
sim.run_simulation()